In [1]:
# -*- coding: utf-8 -*-
from pathlib import Path
import pandas as pd
import numpy as np
# import microdf as mdf
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

In [2]:
# Get the project root directory
project_dir = Path().resolve().parents[0]
# Our data in subdirectories here depending on the step in the pipeline
dir_data = project_dir / "data"
# The directory where we will store the raw, unprocessed data
dir_raw = dir_data / "raw"
# The directory where we will store the processed data
dir_processed = dir_data / "processed"

In [3]:
import cpi
# cpi.update()


## Import population data

In [19]:
import requests
import json
import bs4 as bs

url = "https://pfd.alaska.gov/Division-Info/summary-of-dividend-applications-payments"
# Get the response from the website
response = requests.get(url)
# Parse the html into a soup object
soup = bs.BeautifulSoup(response.text, "html.parser")
# Find the table
table = soup.find("table", {"class": "table table-striped table-bordered"})


In [24]:
soup = bs.BeautifulSoup(response.text, "html.parser")


In [26]:
bs.BeautifulSoup(response.text)

<html><head><title>Request Rejected</title></head><body>The requested URL was rejected. Please consult with your administrator.<br/><br/>Your support ID is: &lt;13320043351595613859&gt;<br/><br/><a href="javascript:history.back();">[Go Back]</a></body></html>

In [29]:
clip=pd.read_clipboard()

In [32]:
import janitor

In [30]:
clip.head()

,Dividend Year,State Population,Applications Received,Applications Paid,Dividend Amount,Percent Change,Total Disbursed Amount
0,2020,"728,903","673,371","630,937",$992.00,-38%,"$625,889,504.00"
1,2019,"732,734","678,738","633,243","$1,606.00",3.7%,"$1,016,988,258.00"
2,2018,"736,239","670,759","639,247","$1,600.00",45.5%,"$1,022,795,200.00"
3,2017,"737,847","670,706","633,005","$1,100.00",7.6%,"$696,305,500.00"
4,2016,"739,828","674,939","638,178","$1,022.00",-50.7%,"$652,217,916.00"


In [38]:
clip = (
clip
.clean_names()
)
clip.head(2)

,dividend_year,state_population,applications_received,applications_paid,dividend_amount,percent_change,total_disbursed_amount
0,2020,"728,903","673,371","630,937",$992.00,-38%,"$625,889,504.00"
1,2019,"732,734","678,738","633,243","$1,606.00",3.7%,"$1,016,988,258.00"


In [48]:
clip.columns[clip.dtypes=='object']

Index(['state_population', 'applications_received', 'applications_paid',
       'dividend_amount', 'percent_change', 'total_disbursed_amount'],
      dtype='object')

In [45]:
# Write clip to dir_raw
# clip.to_csv(dir_raw / "revenue.csv", index=False)

In [79]:
revenue_raw = pd.read_csv(dir_raw / "revenue.csv")
rev = revenue_raw.copy()
rev.head()

,dividend_year,state_population,applications_received,applications_paid,dividend_amount,percent_change,total_disbursed_amount
0,2020,"728,903","673,371","630,937",$992.00,-38%,"$625,889,504.00"
1,2019,"732,734","678,738","633,243","$1,606.00",3.7%,"$1,016,988,258.00"
2,2018,"736,239","670,759","639,247","$1,600.00",45.5%,"$1,022,795,200.00"
3,2017,"737,847","670,706","633,005","$1,100.00",7.6%,"$696,305,500.00"
4,2016,"739,828","674,939","638,178","$1,022.00",-50.7%,"$652,217,916.00"


In [61]:
rev.columns

Index(['dividend_year', 'state_population', 'applications_received',
       'applications_paid', 'dividend_amount', 'percent_change',
       'total_disbursed_amount'],
      dtype='object')

In [ ]:
['state_population', 'applications_received',
       'applications_paid']

In [80]:
# Check if there are any missing values
rev.isnull().sum()
# The percent_change column should have a value of 0 for 1982
rev.loc[rev.dividend_year==1982, 'percent_change']=0

rev.isnull().sum()

dividend_year             0
state_population          0
applications_received     0
applications_paid         0
dividend_amount           0
percent_change            0
total_disbursed_amount    0
dtype: int64

In [85]:
rev=(
    rev
    .transform_columns(
        ["state_population", "applications_received", "applications_paid",],
        function=lambda x: x.str.replace(",", "").astype(int),
        elementwise=False,
    )
    .transform_columns(
        ["dividend_amount", "total_disbursed_amount"],
        function=lambda x: x.str.replace(",", "")
        .str.replace("$", "")
        .astype(float),
        elementwise=False,
    )
    .transform_column(
        "percent_change",
        lambda x: x.str.replace("%", "")
        .astype(float) / 100,
        elementwise=False,
    )
)



/home/fedderw/anaconda3/envs/Maryland-Child-Allowance-Dashboard/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  # This is added back by InteractiveShellApp.init_path()


In [86]:
rev.dtypes

dividend_year               int64
state_population            int64
applications_received       int64
applications_paid           int64
dividend_amount           float64
percent_change            float64
total_disbursed_amount    float64
dtype: object

In [87]:
# total_disbursed_amount / state_population
rev['dividend_per_capita'] = rev.total_disbursed_amount / rev.state_population

In [88]:
rev.tail()

,dividend_year,state_population,applications_received,applications_paid,dividend_amount,percent_change,total_disbursed_amount,dividend_per_capita
34,1986,550700,543858,533315,556.26,0.377,2.966618e+08,538.699477
35,1985,543900,526976,519413,404.00,0.220,2.098429e+08,385.811458
36,1984,524000,494150,482135,331.29,-0.142,1.597265e+08,304.821573
37,1983,499100,469229,458213,386.15,-0.614,1.769389e+08,354.516029
38,1982,464300,487841,470897,1000.00,NaN,4.708970e+08,1014.208486


In [36]:
df = (
    clip
    .clean_names()
    # Remove commas in all values
    .apply(lambda x: x.str.replace(",", ""))
)

df.head()

AttributeError: Can only use .str accessor with string values!

## Wikitables

In [14]:
wiki_tables=pd.read_html("https://en.wikipedia.org/wiki/Alaska_Permanent_Fund")
pfd=wiki_tables[0]
pfd.columns = ['year','amount','amount_cpi_2020']

# regexp: capture '1600.0'0 from '$1,600.00 (dividend was estimated to be $2,700 however it was reduced by legislative action)' 
pfd['amount'] = pfd['amount'].str.replace(r'\$([0-9,]*\.?[0-9]*)', r'\1')
# Delete anything in brackets
pfd['amount'] = pfd['amount'].str.replace(r'\(.*\)', '')
# Delete anything after a space
pfd['amount'] = pfd['amount'].str.replace(r'\s.*', '')

pfd.tail()

/home/fedderw/anaconda3/envs/Maryland-Child-Allowance-Dashboard/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/home/fedderw/anaconda3/envs/Maryland-Child-Allowance-Dashboard/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  
/home/fedderw/anaconda3/envs/Maryland-Child-Allowance-Dashboard/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.


,year,amount,amount_cpi_2020
35,2017,"1,100.00","$1,158.26"
36,2018,"1,600.00","$1,647.85"
37,2019,"1,606.00","$1,625.80"
38,2020,992.00,$992.00
39,2021,1114.00,NaN


In [12]:


# Strip the dollar signs and commas from the amounts
pfd['amount'] = pfd['amount'].str.replace(r'[^\d.]', '')
pfd['amount_cpi_2020'] = pfd['amount_cpi_2020'].str.replace(r'[^\d.]', '')
# Convert the amounts to numeric
pfd.tail()

/home/fedderw/anaconda3/envs/Maryland-Child-Allowance-Dashboard/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/home/fedderw/anaconda3/envs/Maryland-Child-Allowance-Dashboard/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


,year,amount,amount_cpi_2020
35,2017,1100.0024,1158.26
36,2018,1600.0025,1647.85
37,2019,1606.0026,1625.80
38,2020,992.0027,992.00
39,2021,1114.0028,NaN


In [95]:
# Import raw per capita personal income data
pcpi = pd.read_csv(dir_raw / "personal_income.csv")
# Clean column names
pcpi.columns = pcpi.columns.str.lower().str.replace(" ", "_")


# Convert YYYY-MM-DD to datetime
pcpi["date"] = pd.to_datetime(pcpi["date"])

# Rename akpcpi to percapita personal income
pcpi.rename(columns={"akpcpi": "pcpi"}, inplace=True)
print(pcpi.info())
pcpi.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    41 non-null     datetime64[ns]
 1   pcpi    41 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 784.0 bytes
None


,date,pcpi
0,1981-01-01,17091.0
1,1982-01-01,19424.0
2,1983-01-01,19478.0
3,1984-01-01,19701.0
4,1985-01-01,20593.0


In [91]:
# inflation_area= "Urban Alaska"

In [96]:
# inflation_area= "USA"

# index_df=cpi.series.get(area=inflation_area).to_dataframe()
# index_df.head()
# index_df[index_df['year']==1987].info()

CPIObjectDoesNotExist: Object with id USA could not be found

In [89]:

# def apply_cpi(amount, from_year, to_year=2018):
#     return cpi.inflate(amount, from_year, to_year, area="Urban Alaska")

# # Apply CPI to per capita personal income
# pcpi["pcpi_adj"] = pcpi.apply(lambda row: apply_cpi(row["percapita_personal_income"], row["year"]), axis=1) 
# print(pcpi.info())
# pcpi.head()

KeyError: 'year'

In [40]:
# Import raw dividends data
pfd = pd.read_csv(dir_raw / "pfd_amounts.csv")
# Clean column names
pfd.columns = pfd.columns.str.lower().str.replace(" ", "_")
print(pfd.info())
pfd.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   year       39 non-null     int64  
 1   pfd        39 non-null     float64
 2   pfd_notes  4 non-null      object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.0+ KB
None


,year,pfd,pfd_notes
0,1982,1000.00,NaN
1,1983,386.15,NaN
2,1984,331.29,NaN
3,1985,404.00,NaN
4,1986,556.26,NaN


## Fix dates

We want to get the data types right here first of all. We have the years stored in a pandas df column, the format is YYYY stored as an integer. The dividend paid out in October of all years except 2020.

In [93]:
# Import raw dividends data
pfd = pd.read_csv(dir_raw / "pfd_amounts.csv")
# Clean column names
pfd.columns = pfd.columns.str.lower().str.replace(" ", "_")
print(pfd.info())
pfd.head()
# We want to get the data types right here first of all. We have the years stored in a pandas df column, the format is YYYY stored as an integer. The dividend paid out in October of all years except 2020.
# Convert int year to datetime
pfd["year"] = pd.to_datetime(pfd["year"].astype('str'), format="%Y")
# Set the payout date to October 1st if the year is not 2020
pfd["payout_date"] = pfd["year"].apply(lambda x: datetime(x.year, 10, 1) if x.year != 2020 else datetime(x.year, 7, 1))



pfd.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   year       39 non-null     int64  
 1   pfd        39 non-null     float64
 2   pfd_notes  4 non-null      object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.0+ KB
None


,year,pfd,pfd_notes,payout_date
0,1982-01-01,1000.00,NaN,1982-10-01
1,1983-01-01,386.15,NaN,1983-10-01
2,1984-01-01,331.29,NaN,1984-10-01
3,1985-01-01,404.00,NaN,1985-10-01
4,1986-01-01,556.26,NaN,1986-10-01


In [96]:
type(pcpi.date.dt.quarter[0])

numpy.int64

In [100]:
pcpi['amount_t-1'] = pcpi['pcpi'].shift(-1)
pcpi['pfd_year_amount']=pcpi['amount_t-1']*0.75 + pcpi['pcpi']*0.25



In [98]:
print("pcpi shape: ", pcpi.shape)
print(pcpi.dtypes)
print('pfd shape: ', pfd.shape)
print(pfd.dtypes)


pcpi shape:  (41, 4)
date               datetime64[ns]
pcpi                      float64
amount_t-1                float64
pfd_year_amount           float64
dtype: object
pfd shape:  (39, 4)
year           datetime64[ns]
pfd                   float64
pfd_notes              object
payout_date    datetime64[ns]
dtype: object


Index(['year', 'pfd', 'pfd_notes', 'payout_date'], dtype='object')

In [113]:
# Create a plotly line chart that shows multiple lines for the pcpi and pfd
# We'll use the plotly library to do this
# Import the plotly library
import plotly.graph_objects as go

# import cpi as cpi
# Create a figure
fig = go.Figure()

# Add a scatter trace for the pfd
fig.add_trace(go.Scatter(x=pfd["year"], y=pfd["pfd"], name="pfd",
mode="lines+markers+text",
))
# Add a scatter trace for the pcpi
# fig.add_trace(go.Scatter(x=pcpi["date"], y=pcpi["pcpi"], name="pcpi"))
# Add a scatter trace for the pcpi
fig.add_trace(
    go.Scatter(
        x=pcpi["date"],
        y=pcpi["pfd_year_amount"],
        name="pfd year adjusted",
        mode="lines+markers+text",
    )
)
# Add spikelines that show on hover
fig.add_trace(
    go.Scatter(
        x=pcpi["date"],
        y=pcpi["pcpi"],
        name="pcpi",
        mode="lines+markers+text",
        # textposition="top center",
        # text=pcpi["pcpi"],
        hoverinfo="text+y+name",
    )
)


In [117]:
fig = go.Figure()


# Add a scatter trace for the pfd
fig.add_trace(
    go.Scatter(
        x=pfd["year"], 
        y=pfd["pfd"].pct_change(), 
        name="pfd", 
        mode="lines+markers+text",
    )
)
# Add a scatter trace for the pcpi
# fig.add_trace(go.Scatter(x=pcpi["date"], y=pcpi["pcpi"], name="pcpi"))
# Add a scatter trace for the pcpi
fig.add_trace(
    go.Scatter(
        x=pcpi["date"],
        y=pcpi["pfd_year_amount"].pct_change(),
        name="Personal income adjusted",
        mode="lines+markers+text",
    )
)
# # Add spikelines that show on hover
# fig.add_trace(
#     go.Scatter(
#         x=pcpi["date"],
#         y=pcpi["pcpi"].pct_change(),
#         name="pcpi",
#         mode="lines+markers+text",
#         # textposition="top center",
#         # text=pcpi["pcpi"],
#         hoverinfo="text+y+name",
#     )
# )

fig.show()

In [1]:
rev.head()

NameError: name 'rev' is not defined

In [126]:

# VIsualize the same data, but the percent change from the previous year instead of the absolute amount
# Create a figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=pcpi["date"],
        y=pfd['pfd'] / pcpi["pcpi"],
        name="pcpi",
        mode="lines+markers+text",
        # textposition="top center",
        # text=pcpi["pcpi"],
        hoverinfo="text+y+name",
    )
)

fig.add_trace(
    go.Scatter(
        x=pcpi["date"],
        y= rev.dividend_per_capita / pcpi["pcpi"],
        name="Divdend payout per capita",
        mode="lines+markers+text",
        
    ) 
)

# Put the legend below the plot
fig.update_layout(
    legend_orientation="h",
    legend=dict(x=0, y=-0.2),
    title_text="Ratio of dividend payment to personal per capita income",
    xaxis_title="Date",
    yaxis_title="Dividend payout per capita",
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        showline=True,
        ticks="inside",
    )
)

fig.show()

In [108]:
# The best way to compare two time-series with the same units but different scales is to use a logarithmic scale.
# We'll use the logarithmic scale to compare the pcpi and pfd
# Create a figure
fig = go.Figure()
# Add a scatter trace for the pfd
fig.add_trace(go.Scatter(x=pfd['year'], y=pfd['pfd'], name="pfd"))
# Add a scatter trace for the pcpi
fig.add_trace(go.Scatter(x=pcpi['date'], y=pcpi['pcpi'], name="pcpi"))
# Add a scatter trace for the pcpi
fig.add_trace(go.Scatter(x=pcpi['date'], y=pcpi['pfd_year_amount'], name="pfd year adjusted"))
# Add spikelines that show on hover
fig.add_trace(go.Scatter(x=pcpi['date'], y=pcpi['pcpi'], name="pcpi", mode="lines+markers+text", textposition="top center", text=pcpi['pcpi'], hoverinfo="text+y+name"))
# Set the x-axis title
fig.update_xaxes(title_text="Year")
# Set the y-axis title
fig.update_yaxes(title_text="Personal Income")
# Set the title
fig.update_layout(title_text="Personal Income")
# Add a slider to the figure to change the year
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            active=0,
            buttons=list([
                dict(
                    label="Year",
                    method="update",
                ),
                dict(
                    label="Log",
                    method="update",
                    args=[{"yaxis": {"type": "log"}}],
                ),
                
            ]),
        ),
    ],
)


fig.show()